In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import numpy as np
import threading
import concurrent.futures 
import multiprocessing 
import asyncio
import aiohttp
from aiohttp.client import ClientSession
import nest_asyncio
import ssl
import urllib.parse
import time 
import aiohttp_retry
import polars as pl
from sklearn.model_selection import train_test_split
from tqdm import tqdm

Because the data capacity is too large, I split it into many files for storage

1. minify webpage

In [3]:
import os
import glob
import pandas as pd

temp_dir = "DATASETS/details_pyppeteer/phishing"

import minify_html


# Hàm minify HTML
def minify(html):
    if isinstance(html, str):
        return minify_html.minify(html)
    return html  # Nếu không phải là chuỗi, trả lại giá trị gốc

# Hàm để xử lý từng chunk của DataFrame
def process_chunk(df_chunk):
    # Loại bỏ các hàng có giá trị NaN hoặc giá trị "Drop" trong cột Web_details
    df_chunk = df_chunk.dropna(subset=['Web_details'])
    df_chunk = df_chunk[df_chunk['Web_details'] != 'Drop']
    
    df_chunk['Web_details'] = df_chunk['Web_details'].apply(minify)
    return df_chunk

# Đọc từng tệp CSV, xử lý và ghi đè lại
for f in tqdm(glob.glob(f"{temp_dir}/*.csv")):
    print(f"Reading {f}")  # In ra tên file đang xử lý
    df_chunk = pd.read_csv(f)
    
    print(f"Processing {f}")  # In ra tên file đang xử lý
    processed_chunk = process_chunk(df_chunk)
    
    print(f"Saving {f}")  # In ra tên file sau khi xử lý
    processed_chunk.to_csv(f, index=False)  # Ghi đè lại tệp CSV



In [15]:
from bs4 import BeautifulSoup
import re
from collections import OrderedDict

def extract_relevant_content(html_content, url):
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
    except Exception as e:
        logging.error(f"Lỗi khi phân tích HTML: {e}")
        return ""

    # Giới hạn URL ở 100 ký tự đầu tiên
    truncated_url = url[:80]

    # OrderedDict để lưu trữ các phần tử văn bản đã lọc
    content_parts = OrderedDict([(truncated_url, None)])  # Thêm URL bị cắt ngắn

    # Lấy thẻ title nếu có
    if soup.title and soup.title.string:
        content_parts[soup.title.string] = None

    # Lấy các thẻ meta cần thiết
    for meta in soup.find_all('meta'):
        if 'name' in meta.attrs and meta.attrs['name'] in ['description', 'keywords', 'robots']:
            content = meta.attrs.get('content', '').strip()
            if content and content not in content_parts:
                content_parts[content] = None
        elif 'http-equiv' in meta.attrs and meta.attrs['http-equiv'] == 'refresh':
            content = re.split(r';?\s*url=', meta.attrs.get('content', '').strip(), maxsplit=1)[0]
            if content and content not in content_parts:
                content_parts[content] = None
    # Lấy văn bản từ các thẻ h, p, div, span, li, td
    for tag in soup.find_all(['div', 'span', 'li', 'td']):
        text = tag.get_text(separator=' ', strip=True)
        if text:
            sentences = re.split(r'(?<=[.!?]) +', text)
            for sentence in sentences:
                sentence = sentence.strip()
                if sentence and sentence not in content_parts:
                    content_parts[sentence] = None
    # Lấy văn bản từ các thẻ h, p, div, span, li, td
    for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p']):
        text = tag.get_text(separator=' ', strip=True)
        if text:
            sentences = re.split(r'(?<=[.!?]) +', text)
            for sentence in sentences:
                sentence = sentence.strip()
                if sentence and sentence not in content_parts:
                    content_parts[sentence] = None

    # Lấy văn bản bên trong thẻ a
    for tag in soup.find_all('a'):
        text = tag.get_text(strip=True)
        if text and text not in content_parts:
            content_parts[text] = None
        href = tag.get('href', '').strip()
        if href and href not in content_parts:
            content_parts[href] = None

    # Lấy thuộc tính placeholder của thẻ input
    for input_tag in soup.find_all('input'):
        placeholder = input_tag.attrs.get('placeholder', '').strip()
        if placeholder and placeholder not in content_parts:
            content_parts[placeholder] = None

    # Lấy văn bản từ thẻ button
    for button in soup.find_all('button'):
        text = button.get_text(strip=True)
        if text and text not in content_parts:
            content_parts[text] = None

    # Lấy thuộc tính src của script, img, frame, iframe, object, embed
    for tag in soup.find_all(['script', 'img', 'frame', 'iframe', 'object', 'embed']):
        src = tag.get('src', '').strip()
        if src and src not in content_parts:
            content_parts[src] = None

    # Loại bỏ các giá trị None và chuyển đổi OrderedDict thành list
    content_parts = list(content_parts.keys())
    
    combined_content = ' '.join(content_parts)  # Chuyển đổi list thành chuỗi
 

    # Tách chuỗi thành một list các từ
    word_list = combined_content.split()

    # Lấy 2000 từ đầu tiên
    first_2000_words = ' '.join(word_list[:1500])
    
    if first_2000_words.strip() == truncated_url:
        return "Drop"

    return first_2000_words

# # Ví dụ sử dụng
# html_content = df['Web_details'][1]  # Thay bằng nội dung HTML của bạn
# url = df['url'][1]
# relevant_content_string = extract_relevant_content(html_content, url)
# print(html_content)

# print(relevant_content_string)

In [18]:
from tqdm import tqdm
def process_chunk(df):
    for i in tqdm(range(len(df))):
        html_content = df['Web_details'][i]  # Thay bằng nội dung HTML[of]
        url = df['url'][i]
        relevant_content_string = extract_relevant_content(html_content, url)
        df.at[i, 'Web_details'] = relevant_content_string
    return df

In [20]:
save_dir = "DATASETS/clean_details_pyppeteer_1"
os.makedirs(save_dir, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại

for f in glob.glob(f"{temp_dir}/*.csv"):
    print(f"Reading {f}")  # In ra tên file đang xử lý
    df_chunk = pd.read_csv(f)
    df_chunk = df_chunk.drop_duplicates(subset=['Web_details']).reset_index(drop=True)

    print(f"Processing {f}")  # In ra tên file đang xử lý
    processed_chunk = process_chunk(df_chunk)
    processed_chunk = processed_chunk.drop('url', axis=1)
    processed_chunk = processed_chunk[processed_chunk['Web_details'] != 'Drop']
    print(processed_chunk)

    # Lấy tên file từ đường dẫn gốc
    filename = os.path.basename(f)

    # Đường dẫn mới để lưu trong thư mục save_dir
    new_path = os.path.join(save_dir, filename)

    print(f"Saving {new_path}")  # In ra tên file sau khi xử lý
    processed_chunk.to_csv(new_path, index=False)  # Ghi vào file mới

Reading DATASETS/details_pyppeteer/phishing/temp_391000.csv
Processing DATASETS/details_pyppeteer/phishing/temp_391000.csv


100%|██████████████████████████████████████████| 591/591 [00:05<00:00, 117.91it/s]


     label                                        Web_details
0        1  https://www.d455j6j7.blogspot.com Không tìm th...
2        1  https://bicara-se0.blogspot.com/ Bicara SEO ≡ ...
3        1  https://www.ticket-appeal-6f239.web.app Site N...
4        1  https://www.4388gd.com 果冻  果冻  视频 热门 分类 我的 视...
5        1  https://www.muzhnachas-kazan.narod.ru МУЖ НА Ч...
..     ...                                                ...
586      1  https://www.brasserie-osseus.fr Casino 7slots ...
587      1  https://www.mariusidaamel.fkipuika.com 404 Not...
588      1  https://www.tjwanlistone.com KUTASLOT 📍 Link G...
589      1  https://epecpromen1983.blogspot.com/ ...Loadin...
590      1  https://www.comaproductora.com Inicio - COMA P...

[572 rows x 2 columns]
Saving DATASETS/clean_details_pyppeteer_1/temp_391000.csv
Reading DATASETS/details_pyppeteer/phishing/temp_393000.csv
Processing DATASETS/details_pyppeteer/phishing/temp_393000.csv


100%|███████████████████████████████████████████| 578/578 [00:08<00:00, 70.86it/s]


     label                                        Web_details
0        1  https://www.bijouxdesophie.co.cc CC.CC | Store...
1        1  https://www.86bb2.com 警告/Warning *若阁下未满18岁请离开页...
2        1  https://retgvrfdgvfrfd.blogspot.com/ Không tìm...
3        1  https://instagram-login-b1aitours.blogspot.com...
4        1  http://ww25.ms1.edsr.com/?subid1=20240612-1851...
..     ...                                                ...
573      1  https://www.karinsbegravningsbyra.se Karins Be...
574      1           https://www.luna-maria.com 403 Forbidden
575      1  https://www.gitlab.gitlab.test.majesticbank.is...
576      1  https://www.l8xjqelqq.islacyborg.com.ar Forbid...
577      1  https://www.emselec.aaespanol.org Index of / I...

[551 rows x 2 columns]
Saving DATASETS/clean_details_pyppeteer_1/temp_393000.csv
Reading DATASETS/details_pyppeteer/phishing/temp_389000.csv
Processing DATASETS/details_pyppeteer/phishing/temp_389000.csv


...

     label                                        Web_details
0        1  https://www.unifiedhc.salemcorporationnews.com...
1        1  https://roundtaniso1980.blogspot.com/ DRDE 0;U...
2        1  https://dek-anton31.blogspot.com/ Dek-Anton31 ...
3        1  https://www.sport-salute.com Salute e Sport - ...
4        1  https://www.beneficialbooster.com Registrant W...
..     ...                                                ...
578      1  https://www.outdoorfurnituremoldremover.appspo...
579      1  https://fthfhjbn.blogspot.com/ Không tìm thấy ...
580      1  https://cbvvbxcvbxzcgsdfgsdfgcvbcxvbsdfg.blogs...
581      1  https://novaecosolar.com.br/ Nova Eco Energia ...
582      1  https://www.ecuf.info ecuf.info This domain ma...

[563 rows x 2 columns]
Saving DATASETS/clean_details_pyppeteer_1/temp_398000.csv
Reading DATASETS/details_pyppeteer/phishing/temp_386000.csv
Processing DATASETS/details_pyppeteer/phishing/temp_386000.csv


100%|██████████████████████████████████████████| 586/586 [00:05<00:00, 112.49it/s]


     label                                        Web_details
0        1      https://www.farmriiobrasiil.com 403 Forbidden
2        1  https://www.lader43.co.cc CC.CC | Store Launch...
3        1  https://www.emptycjana.000webhostapp.com No su...
4        1  https://www.sadaghfhfdnb888.top Jumia Đăng nhậ...
5        1  https://www.rewweee34.s3.ap-northeast-3.amazon...
..     ...                                                ...
581      1  https://www.pacificcoastcu.com Pacific Coast C...
582      1  https://www.mercanogluinsaat.com.tr Apache HTT...
583      1  https://videohotcuahot.blogspot.com/ Không tìm...
584      1  https://www.cyzx0754.com 论坛 - Powered by Discu...
585      1  http://ww25.karachays.com/?subid1=20240612-175...

[565 rows x 2 columns]
Saving DATASETS/clean_details_pyppeteer_1/temp_386000.csv
Reading DATASETS/details_pyppeteer/phishing/temp_394000.csv
Processing DATASETS/details_pyppeteer/phishing/temp_394000.csv


100%|██████████████████████████████████████████| 611/611 [00:04<00:00, 132.88it/s]

     label                                        Web_details
0        1  https://www.ynsao.godaddysites.com 404 Not Fou...
1        1  https://www.tdsam.com Total Display Solutions ...
2        1  https://www.sgszdfgzxdfgs.blogspot.com Không t...
3        1  https://www.www.pijykxbf.bhdaqznc.crediteurope...
4        1  http://ww25.eejvxnm.zeheyang7.com/?subid1=2024...
..     ...                                                ...
606      1  https://dfgdfghghfgh.blogspot.com/ Không tìm t...
607      1  https://www.suivi-chronopost.com Plesk Obsidia...
608      1  https://sadhgjhgjjklrtyuyiiolghjkghjftryytihu....
609      1  https://www.glestpeopleshou.info glestpeoplesh...
610      1  https://www.tjydlc.com 钱柜娱乐·(中国)手机官网 钱柜娱乐 ◤钱柜娱...

[582 rows x 2 columns]
Saving DATASETS/clean_details_pyppeteer_1/temp_394000.csv
